# Projet maison n° 3

In [ ]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [57]:
import pandas as pd

# names us
def df_names_us():
    df = pd.DataFrame()
    for i in range(1880, 2022):
        data_i = pd.DataFrame()
        data_i = pd.read_csv("yob{:04d}.txt".format(i), sep=",", header=None)
        data_i.columns = ["name", "gender", "births"]
        data_i["year"] = i
        data_i = data_i.iloc[:, [3, 0, 1, 2]]       
        df = pd.concat([df, data_i])
    
    return df
df_names_us()


,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
31532,2021,Zyeire,M,5
31533,2021,Zyel,M,5
31534,2021,Zyian,M,5
31535,2021,Zylar,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [117]:
import numpy as np

# names fr
def df_names_fr():
    ### Read csv and set columns
    df = pd.DataFrame()
    df = pd.read_csv("nat2021.csv", sep=";", keep_default_na=False)#, dtype={"sexe":"uint16", "preusuel":"string", "annais":"uint16", "nombre":"uint32"})
    df.columns = ["gender", "name", "year", "births"]
    df = df.iloc[:, [2, 1, 0, 3]] 
    
    ### Transform data
    df['gender'] = df['gender'].apply(lambda x: "M" if x == 1 else "F")
    
    ### Clean data
    df['name'] = df['name'].apply(lambda x: np.nan if len(str(x)) < 2 else x)
    df['name'] = df['name'].apply(lambda x: np.nan if x == "_PRENOMS_RARES" else x)
    df['year'] = df['year'].apply(lambda x: np.nan if x == "XXXX" else x)

    df = df.dropna()
    df['name'] = df['name'].str.title()
    
    ### Sort values
    df = df.sort_values(['year', 'gender', 'births', 'name'], ascending=[True, True, False, True])
    
    ### reset index
    df = df.reset_index(drop=True)
    
    return df

type(df_names_fr().index)

pandas.core.indexes.range.RangeIndex

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [168]:
# taux de change
def df_taux_change(devises):
    df = pd.DataFrame([], columns=devises)
    
    df = pd.read_csv("Webstat_Export.csv", sep=";", na_values="-")
    ### retrieve index column
    index_col = df["Titre :"]
    
    ### drop col
    for column in df:
        dropColumn = True
        for dev in devises:
            if dev in column:
                dropColumn = False
        if dropColumn:
            df.drop(column, inplace=True, axis=1)
    df["index"] = index_col
    
    ### drop lines
    df.drop([0,1,2,3,4], inplace=True)
    
    ### set index col
    df["index"] = pd.to_datetime(df["index"])
    df = df.set_index('index')
    
    ###rename col
    df.columns = devises
    
    ### reorder on index
    df = df.sort_values(['index'], ascending=[True])
    
    ### clean
    df = df.dropna()
    
    ### datetime index
    
    return df

df_taux_change(["CHF", "GBP", "USD"])


C:\Users\beran\AppData\Local\Temp\ipykernel_16068\4207092148.py:23: UserWarning: Parsing '20/09/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df["index"] = pd.to_datetime(df["index"])
C:\Users\beran\AppData\Local\Temp\ipykernel_16068\4207092148.py:23: UserWarning: Parsing '19/09/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df["index"] = pd.to_datetime(df["index"])
C:\Users\beran\AppData\Local\Temp\ipykernel_16068\4207092148.py:23: UserWarning: Parsing '18/09/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df["index"] = pd.to_datetime(df["index"])
C:\Users\beran\AppData\Local\Temp\ipykernel_16068\4207092148.py:23: UserWarning: Parsing '17/09/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df["index"] = pd.to_datetime(df["index"])
C:\Users\beran\AppDa

,CHF,GBP,USD
index,,,
1999-01-02,"1,6089","0,6904","1,1338"
1999-01-03,"1,593","0,6829","1,0986"
1999-01-04,"1,5957","0,669","1,0772"
1999-01-06,"1,5928","0,6505","1,0434"
1999-01-07,"1,6043","0,6502","1,0264"
...,...,...,...
2022-12-04,"1,0131","0,83455","1,0861"
2022-12-05,"1,0377","0,85293","1,0408"
2022-12-07,"0,9883","0,84823","1,0042"


### Tests

In [169]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        #self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        #self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [170]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [171]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... C:\Users\beran\AppData\Local\Temp\ipykernel_16068\4207092148.py:23: UserWarning: Parsing '20/09/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df["index"] = pd.to_datetime(df["index"])
C:\Users\beran\AppData\Local\Temp\ipykernel_16068\4207092148.py:23: UserWarning: Parsing '19/09/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df["index"] = pd.to_datetime(df["index"])
C:\Users\beran\AppData\Local\Temp\ipykernel_16068\4207092148.py:23: UserWarning: Parsing '18/09/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df["index"] = pd.to_datetime(df["index"])
C:\Users\beran\AppData\Local\Temp\ipykernel_16068\4207092148.py:23: UserWarning: Parsing '17/09/2022' in DD/MM/YYYY fo


FAIL: test_df_taux_change (__main__.Session3Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\beran\AppData\Local\Temp\ipykernel_16068\3288372235.py", line 40, in test_df_taux_change
    self.assertTrue((df.dtypes == float).all())
AssertionError: False is not true

----------------------------------------------------------------------
Ran 3 tests in 10.713s

FAILED (failures=1)
